In [ ]:
***
# Prophage work

### (I) 
### (II) 
### (III) 
***

In [ ]:
>

In [ ]:
import os
import random
import subprocess
from tqdm import tqdm

def run_phageboost(path_klebsiella, path_phageboost, good_strains_file, threads=4):
    """Run PhageBoost on all good strains to predict prophages."""
    good_strains = open(good_strains_file).read().split("\n")
    
    for specie in os.listdir(path_klebsiella):
        if specie.startswith("k") and os.path.isdir(f"{path_klebsiella}/{specie}"):
            strains = os.listdir(f"{path_klebsiella}/{specie}/refseq/bacteria")
            for strain in random.sample(strains, len(strains)):
                if strain in good_strains:
                    path_fna = f"{path_klebsiella}/{specie}/refseq/bacteria/{strain}/prokka_annotation_all/{strain}.fna"
                    path_prophage = f"{path_phageboost}/{strain}"
                    
                    try:
                        os.mkdir(path_prophage)
                    except FileExistsError:
                        print(f"Output for {strain} already exists. Continuing...")
                    
                    if len(os.listdir(path_prophage)) == 0:
                        # Run PhageBoost
                        cmd = f"PhageBoost -f {path_fna} -o {path_prophage} --threads {threads}"
                        subprocess.run(cmd, shell=True)
                        
                        with open(f"{path_prophage}/process_done", "w") as outfile:
                            outfile.write("This strain has been studied")
                    else:
                        print(f"PhageBoost already completed for {strain}")
                        

def extract_phageboost_scores(path_phageboost, output_score_file):
    """Extract prophage prediction scores from PhageBoost results."""
    with open(output_score_file, "w") as outfile:
        for strain in os.listdir(f"{path_phageboost}/phageboost_prediction"):
            if len(os.listdir(f"{path_phageboost}/phageboost_prediction/{strain}")) > 2:
                for file in os.listdir(f"{path_phageboost}/phageboost_prediction/{strain}"):
                    if file.startswith("phages"):
                        info_file = open(f"{path_phageboost}/phageboost_prediction/{strain}/{file}").read().split("\n")[2:]
                        for info in info_file:
                            if info:
                                score = info.split("\t")[5]
                                outfile.write(f"{strain},{score}\n")
                                
                                
def prepare_fastani_input(path_phageboost_pred, path_fastANI, path_phageboot_info, strain_ktype_file, min_score=0.70):
    """Prepare input for FastANI based on prophage predictions with scores >= min_score."""
    # Read strain K-type data
    strain_ktype = {}
    good_strain = open(strain_ktype_file).read().split("\n")
    for info in good_strain:
        if info:
            strain = info.split("\t")[0].strip()
            ktype = info.split("\t")[2].strip()
            strain_ktype[strain] = ktype
    
    # Write the prophage information
    with open(f"{path_phageboot_info}/results_phageboost.{min_score}.tsv", "w") as outfile1:
        outfile1.write("Prophage_name\tProphage_length\tN_genes\tScore\tK_type\n")
        
        for strain in tqdm(os.listdir(path_phageboost_pred)):
            for file in os.listdir(f"{path_phageboost_pred}/{strain}"):
                if file.startswith("phages"):
                    try:
                        resume = pd.read_csv(f"{path_phageboost_pred}/{strain}/{file}", skiprows=1, sep="\t")
                    except Exception as e:
                        print(f"No prophage data for {strain}")
                        continue
                    
                    for _, info in resume.iterrows():
                        if float(info["score"]) >= min_score:
                            prophage_id = info["attributes"].split("phage_id=")[1]
                            prophage_len = int(info["end"]) - int(info["start"])
                            n_genes = info["attributes"].split("n_genes=")[1].split(";")[0]
                            
                            for file2 in os.listdir(f"{path_phageboost_pred}/{strain}"):
                                if prophage_id in file2:
                                    seq = open(f"{path_phageboost_pred}/{strain}/{file2}").read().split("\n")[1]
                                    if not os.path.isfile(f"{path_fastANI}/{strain}__{prophage_id}.fasta"):
                                        with open(f"{path_fastANI}/{strain}__{prophage_id}.fasta", "w") as outfile:
                                            outfile.write(f">{strain}__{prophage_id}\n{seq}")
                                    
                            outfile1.write(f"{strain}__{prophage_id}\t{prophage_len}\t{n_genes}\t{info['score']}\t{strain_ktype[strain]}\n")

                            
def write_fastani_list(path_fastANI, output_list_file):
    """Generate the FastANI list of sequences to be used for comparison."""
    with open(output_list_file, "w") as outfile:
        for file in tqdm(os.listdir(path_fastANI)):
            outfile.write(f"{path_fastANI}/{file}\n")
            
            
def run_fastani(ql_list, rl_list, output_file, threads=40):
    """Run FastANI using the provided query and reference lists."""
    fastani_cmd = (
        f"fastANI --ql {ql_list} --rl {rl_list} -o {output_file} --matrix -t {threads}"
    )
    
    try:
        subprocess.run(fastani_cmd, shell=True, check=True)
        print("FastANI completed successfully!")
    except subprocess.CalledProcessError as e:
        print(f"Error while running FastANI: {e}")
        
        
        
def main():
    # Define paths and files
    path_klebsiella = "/home/conchae/prediction_depolymerase_tropism"
    path_phageboost = "/home/conchae/prediction_depolymerase_tropism/prophage_prediction/phageboost/phageboost_prediction"
    good_strains_file = f"{path_klebsiella}/panacota_pangenome/panacota_pangenome_list.txt"
    output_score_file = f"{path_phageboost}/score_distribution.phageboost.csv"
    path_fastANI = "/home/conchae/prediction_depolymerase_tropism/prophage_prediction/prophage_similarity/phageboost/fastANI_20102022"
    path_phageboot_info = "/home/conchae/prediction_depolymerase_tropism/prophage_prediction/prophage_decipher/phageboost/phageboost_info"
    strain_ktype_file = f"{path_klebsiella}/results_kleborate_count.tsv"
    fastani_list_file = f"{path_phageboot_info}/fastANI_list.20102022.tsv"
    fastani_output_file = f"{path_klebsiella}/prophage_prediction/prophage_similarity/phageboost/fastANI_out_20102022"

    # Step 1: Prophage Annotation
    print("Starting prophage annotation with PhageBoost...")
    run_phageboost(path_klebsiella, path_phageboost, good_strains_file)

    # Step 2: Extract Prediction Scores
    print("Extracting PhageBoost prediction scores...")
    extract_phageboost_scores(path_phageboost, output_score_file)

    # Step 3: Prepare FastANI input files
    print("Preparing FastANI input files...")
    prepare_fastani_input(path_phageboost, path_fastANI, path_phageboot_info, strain_ktype_file)

    # Step 4: Write FastANI list
    print("Writing FastANI list...")
    write_fastani_list(path_fastANI, fastani_list_file)

    # Step 5: Run FastANI
    print("Running FastANI...")
    run_fastani(fastani_list_file, fastani_list_file, fastani_output_file)

    print("Pipeline complete.")

# Execute main function
if __name__ == "__main__":
    main()
